In [ ]:
import copy;
import json;
import requests;
from random import randrange
from bs4 import BeautifulSoup;

def getSoup(inurl):
    r = requests.get(inurl)
    r.encoding = 'utf-8'
    soup=BeautifulSoup(r.text);
    return soup;
#end of getSoup

def gen_page_number_list():
    result=list();
    magical_number=21;#this is found by trying manually
    for i in range(1,magical_number+1):
        sup=getSoup("https://movies.yahoo.com.tw/moviegenre_result.html?genre_id="+str(i)+"&page=1");
        page_count=1;
        if sup.find("div",class_="page_numbox") is not None:
            lis=sup.find("div",class_="page_numbox").find_all("li");
            page_count=int(lis[len(lis)-3].text);
        #print({"id":i,"page_count":page_count});
        result.append({"genre_id":i,"page_count":page_count});
    return result;
#end of gen_page_number_list

def extend_page_dataset(indataset,inurl):
    if inurl in indataset["links"]:
        print("repeated");
        return indataset;
    
    a_data={
        "name":"",
        "link":"",
        "when":"",
        "desc":"",
        "class":[]
    };
    sup=getSoup(inurl);
    info_part=sup.find("div",class_="movie_intro_info_r");
    desc_str=sup.find("span",id="story").text.strip();
    
    when_str="";
    for entry in info_part.find_all("span"):
        if entry.text.startswith("上映日期："):
            when_str=copy.deepcopy(entry.text);
            when_str=when_str[len("上映日期："):];
            break;
    if when_str=="":
        raise Exception("date not found");
        
    class_list=[];
    for entry in info_part.find("div",class_="level_name_box").find_all("a"):
        class_list.append(entry.text.strip());
    
    
    a_data["name"]=info_part.find("h1").text;
    a_data["link"]=inurl;
    a_data["when"]=when_str;
    a_data["desc"]=desc_str;
    a_data["class"]=class_list;
    
    indataset["data"].append(a_data);
    indataset["links"].append(inurl);
    return indataset;
#end of extend_page_dataset


def get_link_of_pages(inurl):
    result=[];
    sup=getSoup(inurl);
    lis=sup.find("ul",class_="release_list").find_all("li");
    for entry in lis:
        url=entry.find("a",class_="gabtn")["href"];
        result.append(url);
    return result;
#end of get_link_of_pages

def dataset_update(dataset={"data":[],"links":[]}):
    for entry in gen_page_number_list():
        for pc in range(1,entry["page_count"]+1):
            url="https://movies.yahoo.com.tw/moviegenre_result.html?"
            url+="genre_id="+str(entry["genre_id"]);
            url+="&page="+str(pc);
            print(url);
            for info_url in get_link_of_pages(url):
                extend_page_dataset(dataset,info_url);
            print(url);
    return dataset;
#end of dataset_update

def homework_2():
    things=dataset_update();
    print("total:"+str(len(things["links"])));
    
    randomnum=randrange(1000);
    filename="./output"+str(randomnum)+".txt"
    print("print to "+filename);
    with open(filename,"x",encoding='utf-8') as ifile:
        ifile.write(json.dumps(things));
    print("done!");
    #with open("./output.txt",encoding='utf-8') as ifile:
    #    print(json.loads(ifile.read()));
    #print("done!");
#end of homework_2


homework_2();

https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=1
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=1
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=2
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=2
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=3
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=3
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=4
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=4
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=5
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=5
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=6
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=6
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=7
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=7
https://movies.yahoo.com.tw/movieg

https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=60
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=61
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=61
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=62
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=62
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=63
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=63
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=64
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=64
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=65
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=65
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=66
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=66
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=67
https://movies.yahoo

https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=119
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=120
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=120
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=121
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=121
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=122
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=122
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=123
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=123
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=124
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=124
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=125
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=125
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=1&page=126
https:

repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=40
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=41
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=41
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=42
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=42
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=43
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=43
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=44
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=44
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=2&page=45
repeated
repeated
repeated
repeated
repeated
repeated
repeated
htt

repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=21
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=22
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=22
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=23
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=23
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=24
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=24
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=25
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=25
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=3&page=26
repeated
repeated
rep

...

repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=4
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=5
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=5
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=6
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=6
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=7
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=7
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page=8
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://

repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=9
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=10
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=10
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=11
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=11
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=12
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=12
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=13
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=13
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=14
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_

repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=53
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=54
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=54
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=55
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=55
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=56
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=56
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=57
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=57
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page=58
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre

repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=6
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=7
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=7
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=8
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=8
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=9
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=9
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=10
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=10
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=14&page=11
repeated
repeated
repeated
repeated
repeated
repeated


repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=17
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=18
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=18
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=19
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=19
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=20
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=20
https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=15&page=21
repeated
repeated
repeated
repeated
repeated
repeated
repeated
repeated
https://m

In [127]:
with open("./output412.txt",encoding='utf-8') as ifile:
    print(json.loads(ifile.read()));
print("done!");

{'data': [{'name': 'Thunderbolt Fantasy 西幽玹歌', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/Thunderbolt-Fantasy-%E8%A5%BF%E5%B9%BD%E7%8E%B9%E6%AD%8C-thunderbolt-fantasy-10244', 'when': '2019-10-25', 'desc': '生來便懷有異能歌聲的少年浪巫謠，自小便跟在隱遁雪山的盲眼母親身邊，接受經年累月的苛刻訓練。\n\xa0\n母親懷著野心，想將兒子的歌聲鍛鍊成天下無雙，然後送入宮廷。然而過於苛刻的訓練卻招致了不幸的事故，母親在浪巫謠眼前斷送了性命。\n\xa0\n失去照顧者後，浪巫謠成了流浪之身。但他的異能卻總被無情的人們利用，成為他們滿足慾望的道具，漸漸消磨著少年的心靈。\n\xa0\n儘管如此，浪巫謠罕見的歌聲終於吸引到了西幽皇女，因而得到了過去母親所夢想的飛黃騰達。但是等待著他的，卻是成為執政者玩物、賭上性命與其他樂師進行演奏競賽的殘忍遊戲。\n\xa0\n就在某一天，浪巫謠聽聞有一名在西幽各地奪取魔劍、占為己有的大罪人──「啖劍太歲」。而啖劍太歲的下一個目標，正是皇帝藏在宮中的聖劍。\n\xa0\n【關於電影】\n\xa0\n在臺灣可以說是「無人不知、無人不曉」，從大人到小孩都非常喜愛的木偶劇「布袋戲」。 本次作品由對布袋戲深深著迷的Nitroplus「虛淵玄」擔任故事原著・劇本・總監修，與臺灣布袋戲中最具知名度及以其高品質製作聞名的「霹靂國際多媒體」（日文簡稱：霹靂社）合作， 以此奇蹟般的組合所完成的日本及臺灣地區共同企劃之影像作品。\n\xa0\n於2018年12月24日（一）播放TV版第二季《Thunderbolt Fantasy 東離劍遊紀２》最後一集， 在那之後隨即發表「第三季製作決定！」的情報， 並且在3月22日（五）公佈第二季外傳《Thunderbolt Fantasy 西幽玹歌》正在進行製作的消息。\xa0\n\xa0\n2019年10月25日（五）於戲院上映的《Thunderbolt Fantasy 西幽玹歌》，以活躍於TV版第二季的「浪巫謠」為主角，講述他的過去、以及他在西幽發生的故事，為Nitroplus「虛淵玄」擔任故事原著・劇本